<a href="https://colab.research.google.com/github/springboardmentor891v/Email_Assistant_Using_LangGraph/blob/manoj_preetham/Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.9 MB/s eta 0:00:00


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915502763627-c2ilmm6t0n042r0lbjg7mjr0f0mmqehn.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A40801%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=9vCmMtEocfxcX50DRXaTfNenBHqUxo&access_type=offline


KeyboardInterrupt: 

In [ ]:
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('/content/token.json'):
        creds = Credentials.from_authorized_user_file(
            '/content/token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/content/credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # 🔑 KEY LINE
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n➡️ Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

    service = build('gmail', 'v1', credentials=creds)
    return service
